<a href="https://colab.research.google.com/github/stephenbeckr/convex-optimization-class/blob/master/Demos/RPCA_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case study: RPCA

In-class challenge: how many different algorithms could you use to solve the RPCA problem?

APPM 5630 Adv Convex Optimization, Spring 2023, Professor Stephen Becker

**See companion file `RPCA_case_study_solutions.ipynb` for some solutions**

In [ ]:
import numpy as np
from scipy import linalg
from numpy.linalg import norm
from numpy.random import default_rng
import cvxpy as cvx
%reload_ext autoreload
%autoreload 2
!wget -nv 'https://github.com/stephenbeckr/convex-optimization-class/raw/master/utilities/firstOrderMethods.py'
!wget -nv 'https://github.com/stephenbeckr/convex-optimization-class/raw/master/utilities/secondOrderMethods.py'
from firstOrderMethods import gradientDescent, lassoSolver, createTestProblem, runAllTestProblems, DouglasRachford, bookkeeper
from secondOrderMethods import NewtonsMethod

## The Robust PCA (RPCA) problem
$$\min_{L,S} \;\frac12\|L+S-Y\|_F^2 + \lambda_L\|L\|_* + \lambda_S\|S\|_1$$
where $L,S$ are matrices of the same size; $Y$ is the target matrix that we wish to approximately decompose, $Y \approx L + S$, into a low-rank part $L$ and a sparse part $S$

In [ ]:
rng = default_rng(1)
m,n = 7,8

# for use later
def vec(L,S):
    return np.concatenate( (L.ravel(), S.ravel() ))
def mat( LS ):
    L = LS[:m*n].reshape( (m,n ))
    S = LS[m*n:].reshape( (m,n ))
    return L, S

# Generate a "signal". Need not be solution to optimization problem
rr  = 4
LL  = rng.standard_normal( (m,rr) ) @ rng.standard_normal( (rr,n) ) # low-rank
SS  = rng.standard_normal( (m,n) )
SS[ np.abs(SS) < np.quantile( np.abs(SS), .9 ) ] = 0  # sparse
Y   = LL + SS + .01*rng.standard_normal( (m,n) ) # Noisy observations

# with np.printoptions(precision=3, suppress=True):
#     print(SS)

### Solve in CVXPY for reference solution

This is one way to solve: the $\ell_1$ and nuclear norms can be written in terms of epigraphs of standard cones and such, and then solved via standard methods (IPM, ADMM)

In [ ]:
# Solve in cvxpy
L     = cvx.Variable( (m,n) )
S     = cvx.Variable( (m,n) )
lambdaL = 3e-3
lambdaS = 1.7e-3

def prox_l1( X, t, lambdaS ):
    return np.sign(X)*np.maximum( 0, np.fabs(X) - lambdaS*t )

def prox_nuclearNorm( X, t, lambdaL ):
   U, s, Vh = linalg.svd(X, full_matrices=False )
   s = prox_l1( s, t, lambdaL )
   return U@( s.reshape((-1,1))*Vh )

def objective_vec( LSvec ):
    L,S = mat(LSvec)
    resid = norm( L+S - Y)
    return resid**2/2 + lambdaL*np.sum( linalg.svdvals(L) ) + lambdaS*norm( S.ravel(), ord=1)
obj   = cvx.Minimize( cvx.sum_squares(L+S-Y)/2 + lambdaL*cvx.norm(L, "nuc") + lambdaS*cvx.pnorm(S,p=1) )
prob  = cvx.Problem(obj)
highPrecision = {'solver':cvx.CVXOPT,'max_iters':400,'abstol':1e-11,'reltol':1e-11}
prob.solve(**highPrecision)
L_CVX = L.value 
S_CVX = S.value 
fTrue = prob.value

with np.printoptions(precision=3, suppress=True):
    print( linalg.svdvals( L_CVX ))
    print( linalg.svdvals( LL ) )
    print(SS)
    print(S_CVX)

[10.374  4.849  2.435  1.788  0.466  0.314  0.   ]
[10.373  4.687  2.211  1.507  0.     0.     0.   ]
[[ 0.     0.     0.     0.     0.     0.     0.     0.   ]
 [-1.649  0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.    -1.482  0.     0.     0.     0.    -1.631  0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.    -2.251  0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.    -1.514]
 [ 1.753  0.     0.     0.     0.     0.     0.     0.   ]]
[[-0.    -0.    -0.     0.    -0.     0.     0.    -0.   ]
 [-0.54   0.     0.     0.     0.     0.    -0.     0.   ]
 [ 0.    -0.61  -0.     0.    -0.     0.    -0.666 -0.   ]
 [-0.    -0.    -0.    -0.     0.    -0.671 -0.    -0.   ]
 [ 0.     0.    -0.    -1.694 -0.     0.    -0.     0.   ]
 [ 0.     0.    -0.    -0.     0.     0.    -0.    -0.912]
 [ 0.     0.    -0.188  0.     0.    -0.    -0.     0.   ]]


# The algorithms